### Based on: https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/ and https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/ and https://github.com/stanfordnlp/GloVe

In [4]:
#import time
import datetime
import string
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

import numpy as np

from random import randint
from keras.preprocessing.sequence import pad_sequences

def get_time( output=True ):
    
    temp = time.time()
    if output:
        now = datetime.datetime.now()
        print( now.strftime( "%Y.%m.%d %H:%M" ) )
        
    return temp

foo = get_time()

def print_time( start_time, end_time, interval="seconds" ):
    
    if interval == "hours":
        print ( "Time to process: [%s] hours" % ( str( ( end_time - start_time ) / 60 / 60 ) ) )
    else:
        print ( "Time to process: [%s] seconds" % ( str( end_time - start_time ) ) )

print_time( 0, 1 )

2018.05.21 10:47
Time to process: [1] seconds


In [5]:
# load doc into memory
def load_doc( filename ):
    
    # open the file as read only
    file = open( filename, 'r' )
    # read all text
    text = file.read()
    # close the file
    file.close()
    
    return text

# load document
#in_filename = "../texts/alice-in-wonderland.txt"
#in_filename = "../texts/dr-zeuss-compilation.txt"
in_filename = "../texts/trump-speeches.txt"
doc = load_doc( in_filename )
print( doc[ :200 ] )

﻿SPEECH 1


...Thank you so much.  That's so nice.  Isn't he a great guy.  He doesn't get a fair press; he doesn't get it.  It's just not fair.  And I have to tell you I'm here, and very strongly here


In [13]:
# my_punctuation = string.punctuation
# print( type( my_punctuation ) )
# print( my_punctuation )
my_punctuation = '"#$%&\'()*+,-/:;<=>@[\\]^_`{|}~'
my_punctuation

'"#$%&\'()*+,-/:;<=>@[\\]^_`{|}~'

In [18]:
# turn a doc into clean tokens
def clean_doc( doc, to_lower=True ):
    
    # replace '--' with a space ' '
    doc = doc.replace( '--', ' ' )
    # replace sentence simple sentence boundaries w/ unique token/markers
    doc = doc.replace( '. ', ' endperiod ' )
    doc = doc.replace( '! ', ' endexclamation ' )
    doc = doc.replace( '? ', ' endquestion ' )
    doc = doc.replace( ', ', ' pausecomma ' )
    doc = doc.replace( ': ', ' pausecolon ' )
    doc = doc.replace( '; ', ' pausesemicolon ' )
    
    # split into tokens by white space
    tokens = doc.split()
    
    # remove punctuation from each token
    table = str.maketrans( '', '', string.punctuation ) # will strip all .?!,:; that don't fit replace expr above.
    #table = str.maketrans( '', '', my_punctuation )
    tokens = [ w.translate( table ) for w in tokens ]
    
    # remove remaining tokens that are not alphabetic
    if to_lower:
        tokens = [ word for word in tokens if word.isalpha() ]
    
    # make lower case
    tokens = [ word.lower() for word in tokens ] 
    
    return tokens

In [19]:
# clean document
tokens = clean_doc( doc )
tokens_unique = list( set( tokens ) )
print( tokens[ :200 ] )
print( 'Total Tokens: %d' % len( tokens ) )
print( 'Unique Tokens: %d' % len( tokens_unique ) )

['thank', 'you', 'so', 'much', 'endperiod', 'thats', 'so', 'nice', 'endperiod', 'isnt', 'he', 'a', 'great', 'guy', 'endperiod', 'he', 'doesnt', 'get', 'a', 'fair', 'press', 'pausesemicolon', 'he', 'doesnt', 'get', 'it', 'endperiod', 'its', 'just', 'not', 'fair', 'endperiod', 'and', 'i', 'have', 'to', 'tell', 'you', 'im', 'here', 'pausecomma', 'and', 'very', 'strongly', 'here', 'pausecomma', 'because', 'i', 'have', 'great', 'respect', 'for', 'steve', 'king', 'and', 'have', 'great', 'respect', 'likewise', 'for', 'citizens', 'united', 'pausecomma', 'david', 'and', 'everybody', 'pausecomma', 'and', 'tremendous', 'resect', 'for', 'the', 'tea', 'party', 'endperiod', 'also', 'pausecomma', 'also', 'the', 'people', 'of', 'iowa', 'endperiod', 'they', 'have', 'something', 'in', 'common', 'endperiod', 'hardworking', 'people', 'endperiod', 'they', 'want', 'to', 'work', 'pausecomma', 'they', 'want', 'to', 'make', 'the', 'country', 'great', 'endperiod', 'i', 'love', 'the', 'people', 'of', 'iowa', 'en

In [20]:
# organize into sequences of tokens
sequence_len = 50 + 1
sequences = list()

for i in range( sequence_len, len( tokens ) ):
    
    # select sequence of tokens
    seq = tokens[ i - sequence_len:i ]
    
    # convert into a line
    line = ' '.join( seq )
    
    # store
    sequences.append( line )
    
print( 'Total Sequences: %d' % len( sequences ) )


Total Sequences: 176937


In [21]:
# save tokens to file, one dialog per line
def save_doc( lines, filename ):
    
    data = '\n'.join( lines )
    file = open( filename, 'w' )
    file.write( data )
    file.close()

In [22]:
# save sequences to file
#out_filename = "../texts/dr-zeuss-compilation-sequences.txt"
out_filename = "../texts/trump-speeches-sequences-02.txt"
save_doc( sequences, out_filename )

In [23]:
#in_filename = "../texts/dr-zeuss-compilation-sequences.txt"
in_filename = "../texts/trump-speeches-sequences-02.txt"
doc = load_doc( in_filename )
lines = doc.split( '\n' )
lines[ 0:10 ]

['thank you so much endperiod thats so nice endperiod isnt he a great guy endperiod he doesnt get a fair press pausesemicolon he doesnt get it endperiod its just not fair endperiod and i have to tell you im here pausecomma and very strongly here pausecomma because i have great respect',
 'you so much endperiod thats so nice endperiod isnt he a great guy endperiod he doesnt get a fair press pausesemicolon he doesnt get it endperiod its just not fair endperiod and i have to tell you im here pausecomma and very strongly here pausecomma because i have great respect for',
 'so much endperiod thats so nice endperiod isnt he a great guy endperiod he doesnt get a fair press pausesemicolon he doesnt get it endperiod its just not fair endperiod and i have to tell you im here pausecomma and very strongly here pausecomma because i have great respect for steve',
 'much endperiod thats so nice endperiod isnt he a great guy endperiod he doesnt get a fair press pausesemicolon he doesnt get it endperio

## Convert Words to Index Values

In [24]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts( lines )
sequences = tokenizer.texts_to_sequences( lines )

In [25]:
print( len( sequences[ 0 ] ) == sequence_len )
print( len( sequences ) )

True
176937


In [26]:
print( len( tokenizer.word_index ) )
print( type( tokenizer.word_index ) )
print( tokenizer.word_index[ "pausecomma" ] )

5848
<class 'dict'>
2


In [27]:
# vocabulary size
vocab_size = len( tokenizer.word_index ) + 1
vocab_size

5849

In [28]:
# separate into input and output: for now it's 50 words input and 1 word output
sequences = np.array( sequences )
X = sequences[ :,:-1 ] # all rows, from word 0 up to, but not including, the last word
y = sequences[ :,-1 ]  # all rows, last word only
y = to_categorical( y, num_classes=vocab_size )
seq_length = X.shape[ 1 ]
seq_length

50

## Load and Filter GloVe Data

In [29]:
# load the whole embedding into memory
embeddings_index = dict()
embeddings_dimension = 300 #must be 50, 100, 200, 300
glove = open( "../glove/glove.6B." + str( embeddings_dimension ) + "d.txt" )

for line in glove:
    
    values = line.split()
    # 1st string is word...
    word = values[ 0 ]
    
    if word in tokens_unique:
        
        # ...the rest are coefficients
        coefs = np.asarray( values[ 1: ], dtype='float32' )
        embeddings_index[ word ] = coefs
        print( "*", end="" )
    
glove.close()
print( '\nLoaded %s word vectors.' % len( embeddings_index ) )
print( '\nWords not found %d.' % ( len( tokenizer.word_index ) - len( embeddings_index ) ) )

****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

### Transform into Matrix That Maps Coefs by Index

In [30]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros( ( vocab_size, embeddings_dimension ) )
missing_words = []

# we need this to create empty coefficients array
dummy_shape = embeddings_index[ "the" ].shape

for word, i in tokenizer.word_index.items():
    
    embedding_vector = embeddings_index.get( word )
    
    # not all words in our token list are in the wikipedia 400K set!
    if embedding_vector is None:
        
        # report and create empty coefficients array
        missing_words.append( word )
        embedding_vector = np.zeros( dummy_shape )
        
    embedding_matrix[ i ] = embedding_vector
    
print( len( missing_words ) )
missing_words

204


['endperiod',
 'pausecomma',
 'endquestion',
 'pausesemicolon',
 'pausecolon',
 'endexclamation',
 'selffunding',
 'theyve',
 'youve',
 'beada',
 'selfinspect',
 'theyll',
 'twoway',
 'onethird',
 'reince',
 'lowenergy',
 'bluecollar',
 'ayeyeye',
 'fbomb',
 'obamaclinton',
 'maralago',
 'bigly',
 'sixyear',
 'escavators',
 'nationbuilding',
 'africanamerican',
 'byebye',
 'selfinspection',
 'offmike',
 'antiwoman',
 'intelligencegathering',
 'thatand',
 'africanamericans',
 'donaldjtrumpcom',
 'mexicanamerican',
 'hardhitting',
 'braggadocious',
 'selfpolice',
 'komatsus',
 'nobodys',
 'taxexempt',
 'indianas',
 'fiveforone',
 'oreilly',
 'resect',
 'everyones',
 'theyd',
 'cetain',
 'romneycare',
 'oneyard',
 'threefooter',
 'werent',
 'clearsighted',
 'nationstate',
 'goodsized',
 'airconditioner',
 'disastertrump',
 'nogood',
 'peopletrump',
 'exampletrump',
 'brandnew',
 'oldfashioned',
 'wellover',
 'trilliontrump',
 'jobproducer',
 'truthteller',
 'expresident',
 'middleincome',

In [31]:
# confirm visually that 
print( len( embedding_matrix[ 0 ] ) )
print( sum( embedding_matrix[ 0 ] ) )
empty_coefficients_count = 0

for i in range( len( embedding_matrix ) ):
    if sum( embedding_matrix[ i ] ) == 0:
        empty_coefficients_count += 1
        
empty_coefficients_count

300
0.0


205

## Define Model

In [17]:
import keras
print( keras.__version__ )

import tensorflow as tf
print( tf.__version__ )

2.1.3
1.4.1


In [32]:
# define model
model = Sequential()

# now using a pre-trained, non-trainable embedding from glove's wiki analysis
model.add( Embedding( vocab_size, embeddings_dimension, weights=[embedding_matrix], input_length=seq_length, trainable=True ) )
model.add( LSTM( seq_length * 2, return_sequences=True ) )
model.add( LSTM( seq_length * 2 ) )
model.add( Dense( seq_length * 2, activation='relu' ) )

# fixed TypeError below, downgraded keras from 2.1.5 to 2.1.3: https://github.com/keras-team/keras/issues/9621
# TypeError: softmax() got an unexpected keyword argument 'axis'
model.add( Dense( vocab_size, activation='softmax' ) )

print( model.summary() )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 300)           1754700   
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           160400    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 5849)              590749    
Total params: 2,596,349
Trainable params: 2,596,349
Non-trainable params: 0
_________________________________________________________________
None


## Fit the Model

In [34]:
# calc batch size
print( len( sequences ) / 128 )
print( len( sequences ) / 1028 )
# Was:
#batch_size = 128
batch_size = 1028


1382.3203125
172.11770428015564


In [58]:
start_time = get_time()
# compile model
model.compile( loss='categorical_crossentropy', optimizer='adam', metrics=[ 'accuracy' ] )
# fit model
model.fit( X, y, batch_size=batch_size, epochs=300 )
end_time = get_time()
print_time( start_time, end_time )

2018.05.21 13:43
Epoch 1/300
176937/176937 [==============================] - 27s 153us/step - loss: 2.4837 - acc: 0.4610
Epoch 2/300
176937/176937 [==============================] - 27s 151us/step - loss: 2.4333 - acc: 0.4694
Epoch 3/300
176937/176937 [==============================] - 27s 152us/step - loss: 2.4164 - acc: 0.4738
Epoch 4/300
176937/176937 [==============================] - 27s 152us/step - loss: 2.4068 - acc: 0.4743
Epoch 5/300
176937/176937 [==============================] - 27s 153us/step - loss: 2.3940 - acc: 0.4763
Epoch 6/300
176937/176937 [==============================] - 27s 153us/step - loss: 2.3905 - acc: 0.4772
Epoch 7/300
176937/176937 [==============================] - 27s 153us/step - loss: 2.3739 - acc: 0.4802
Epoch 8/300
176937/176937 [==============================] - 27s 153us/step - loss: 2.3636 - acc: 0.4826
Epoch 9/300
176937/176937 [==============================] - 27s 153us/step - loss: 2.3536 - acc: 0.4836
Epoch 10/300
176937/176937 [==========

176937/176937 [==============================] - 27s 153us/step - loss: 1.8108 - acc: 0.5861
Epoch 79/300
176937/176937 [==============================] - 27s 153us/step - loss: 1.8086 - acc: 0.5881
Epoch 80/300
176937/176937 [==============================] - 27s 153us/step - loss: 1.8132 - acc: 0.5857
Epoch 81/300
176937/176937 [==============================] - 27s 153us/step - loss: 1.8100 - acc: 0.5882
Epoch 82/300
176937/176937 [==============================] - 27s 153us/step - loss: 1.8139 - acc: 0.5847
Epoch 83/300
176937/176937 [==============================] - 27s 153us/step - loss: 1.7986 - acc: 0.5888
Epoch 84/300
176937/176937 [==============================] - 27s 153us/step - loss: 1.7937 - acc: 0.5888
Epoch 85/300
176937/176937 [==============================] - 27s 153us/step - loss: 1.7723 - acc: 0.5953
Epoch 86/300
176937/176937 [==============================] - 27s 153us/step - loss: 1.7616 - acc: 0.5983
Epoch 87/300
176937/176937 [==============================]

176937/176937 [==============================] - 27s 154us/step - loss: 1.4649 - acc: 0.6568
Epoch 155/300
176937/176937 [==============================] - 27s 154us/step - loss: 1.4514 - acc: 0.6608
Epoch 156/300
176937/176937 [==============================] - 27s 153us/step - loss: 1.4589 - acc: 0.6583
Epoch 157/300
176937/176937 [==============================] - 27s 153us/step - loss: 1.4712 - acc: 0.6544
Epoch 158/300
176937/176937 [==============================] - 27s 153us/step - loss: 1.4483 - acc: 0.6595
Epoch 159/300
176937/176937 [==============================] - 27s 153us/step - loss: 1.4398 - acc: 0.6636
Epoch 160/300
176937/176937 [==============================] - 27s 154us/step - loss: 1.4418 - acc: 0.6622
Epoch 161/300
176937/176937 [==============================] - 27s 153us/step - loss: 1.4583 - acc: 0.6580
Epoch 162/300
176937/176937 [==============================] - 27s 153us/step - loss: 1.4762 - acc: 0.6532
Epoch 163/300
176937/176937 [======================

176937/176937 [==============================] - 27s 154us/step - loss: 1.2803 - acc: 0.6944
Epoch 231/300
176937/176937 [==============================] - 27s 154us/step - loss: 1.2576 - acc: 0.7010
Epoch 232/300
176937/176937 [==============================] - 27s 154us/step - loss: 1.2498 - acc: 0.7030
Epoch 233/300
176937/176937 [==============================] - 27s 154us/step - loss: 1.2479 - acc: 0.7023
Epoch 234/300
176937/176937 [==============================] - 27s 154us/step - loss: 1.2412 - acc: 0.7033
Epoch 235/300
176937/176937 [==============================] - 27s 154us/step - loss: 1.2585 - acc: 0.7006
Epoch 236/300
176937/176937 [==============================] - 27s 154us/step - loss: 1.2509 - acc: 0.7012
Epoch 237/300
176937/176937 [==============================] - 27s 154us/step - loss: 1.2453 - acc: 0.7027
Epoch 238/300
176937/176937 [==============================] - 27s 154us/step - loss: 1.2258 - acc: 0.7074
Epoch 239/300
176937/176937 [======================

In [59]:
# save the model to file
model.save( "models/trump-speeches-03.keras" )

# save the tokenizer
dump( tokenizer, open( "tokenizers/trump-speeches-03.pkl", 'wb' ) )

# save embedding_matrix based on wiki embeddings, complete w/ missing coefficients array dummies
dump( embedding_matrix, open( "embeddings/trump-speeches-03.glove", 'wb' ) )


## Use The Model to Generate Text

In [38]:
seq_length = len( lines[ 0 ].split() ) - 1
seq_length

50

## Build Encoded Punctuation to Punctuation Dictionary

In [48]:
punctuation_dict = {}
punctuation_dict[ "endperiod" ] = "."
punctuation_dict[ "endquestion" ] = "?"
punctuation_dict[ "endexclamation" ] = "!"
punctuation_dict[ "pausecomma" ] = ","
punctuation_dict[ "pausecolon" ] = ":"
punctuation_dict[ "pausesemicolon" ] = ";"


In [51]:
punctuation_dict.get( "foo", "bar" )

'bar'

In [52]:
def generate_seq( model, tokenizer, seq_length, seed_text, n_words ):
    
    result = list()
    in_text = seed_text
    
    # generate a fixed number of words
    for _ in range( n_words ):
        
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences( [ in_text ] )[ 0 ] 
        
        # truncate sequences to a fixed length
        encoded = pad_sequences( [ encoded ], maxlen=seq_length, truncating='pre' ) 
        
        # predict probabilities for each word
        yhat = model.predict_classes( encoded, verbose=0 )
        
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break 
                
        # append to input
        in_text += ' ' + out_word
        
#         # create output, conditionally
#         if out_word in punctuation_dict:
#             out_word = punctuation_dict.get( out_word, out_word )
        #result.append( out_word )
        result.append( punctuation_dict.get( out_word, out_word ) )
        
    return ' '.join( result )

In [56]:
def reformat_punctuation( doc ):
    
    doc = doc.replace( ' . ', '. ' )
    doc = doc.replace( ' ! ', '! ' )
    doc = doc.replace( ' ? ', '? ' )
    doc = doc.replace( ' , ', ', ' )
    doc = doc.replace( ' : ', ': ' )
    doc = doc.replace( ' ; ', '; ' )
    
    return doc

In [57]:
# select a seed text
seed_text = lines[ randint( 0, len( lines ) ) ]
print( seed_text + '...\n' )
#print( len( seed_text.split( " " ) ) )

# generate new text
generated = generate_seq( model, tokenizer, seq_length, seed_text, 50 )
print( "..." + reformat_punctuation( generated ) )

that has tremendous personal energy to get us back on track endperiod you do that when you have that i think marco is highly overrated endperiod highly overrated endperiod he have it and all you have to do is look at his stance on things jeb he lacks the quality that...

...you will be great anymore. we have to do it and they said, they said, i think going to do it. i mean, not going to happen. going to be able to do well, going to be the wall, folks. going


In [41]:
def generate_seq_word_by_word( model, tokenizer, seq_length, seed_text, n_words ):
    
    print( "...", end='' )
    #result = list()
    in_text = seed_text
    
    # generate a fixed number of words
    for _ in range( n_words ):
        
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences( [ in_text ] )[ 0 ] 
        
        # truncate sequences to a fixed length
        encoded = pad_sequences( [ encoded ], maxlen=seq_length, truncating='pre' ) 
        
        # predict probabilities for each word
        yhat = model.predict_classes( encoded, verbose=0 )
        
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                print( word, end=' ' )
                break 
                
        # append to input for next iteration
        in_text += ' ' + out_word

In [42]:
# select a seed text
seed_text = lines[ randint( 0, len( lines ) ) ]
print( seed_text + '...\n' )

# generate new text
generate_seq_word_by_word( model, tokenizer, seq_length, seed_text, 25 )

take two minutes to vote now pausecomma yours is a little more complicated because the caucus system is a little bit different endperiod a very interesting system endperiod but it is different what i have to say about iowa endperiod very important so you know a movement to move iowa to...

...the united states endperiod we have a lot of mine that are going through the border pausecomma and going to be stronger than ever before 

In [50]:
my_input = input()

# generate new text
generate_seq_word_by_word( model, tokenizer, seq_length, my_input, 25 )

trump walked down
...a fortune up your company so trump is so you sit right and i just want to skip iowa i was and he said what 